In [ ]:
import openai

openai.api_key = "your-api-key"

response = openai.Completion.create(
    engine="gpt-3.5-turbo",  # Choose the correct model
    prompt="Summarize the following descriptions: {}".format(descriptions),
    max_tokens=150,  # Adjust based on how concise you want the summary to be
)

print(response.choices[0].text.strip())

In [6]:
import pandas as pd
import os

In [11]:
current_directory = os.getcwd()

parent_directory = os.path.dirname(current_directory)

DATA_PATH = os.path.join(parent_directory, "data")
DATA_TOURNAMENT = os.path.join(DATA_PATH, "tournament_hints_data.parquet")

DATA_TOURNAMENT = os.path.join(DATA_PATH, "tournament_hints_data.parquet")
DATA_BUSINESS = os.path.join(DATA_PATH, "Business category taxonomy.xlsx")
DATA_NAICS = os.path.join(
    DATA_PATH,"naics3_tournament_taxonomy.xlsx",
)


print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

Current Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament/marian
Parent Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament


In [12]:
df_naics = pd.read_excel(DATA_NAICS)
df_naics

,naics_code,naics_label,description
0,111,Crop Production,Industries in the Crop Production subsector gr...
1,112,Animal Production and Aquaculture,Industries in the Animal Production and Aquacu...
2,113,Forestry and Logging,Industries in the Forestry and Logging subsect...
3,114,"Fishing, Hunting and Trapping","Industries in the Fishing, Hunting and Trappin..."
4,115,Support Activities for Agriculture and Forestry,Industries in the Support Activities for Agric...
...,...,...,...
91,924,Administration of Environmental Quality Programs,The Administration of Environmental Quality Pr...
92,925,"Administration of Housing Programs, Urban Plan...","The Administration of Housing Programs, Urban ..."
93,926,Administration of Economic Programs,The Administration of Economic Programs subsec...
94,927,Space Research and Technology,The Space Research and Technology subsector co...


In [14]:
df_business = pd.read_excel(DATA_BUSINESS)
df_business

,label,description
0,ATVs Dealers & Services,Businesses categorized under ATVs Dealers & Se...
1,Abortion Clinics,Abortion Clinics provide medical services rela...
2,Accounting & Bookkeeping Services,Accounting & Bookkeeping Services encompass a ...
3,Acupuncture clinic,An Acupuncture clinic is a healthcare facility...
4,Adhesives & Sealants,Adhesives & Sealants encompass a diverse range...
...,...,...
598,Work Clothing & Protection Equipment,Work Clothing & Protection Equipment pertains ...
599,Writers & Copywriters,Writers & Copywriters are skilled professional...
600,YMCA Camps,YMCA Camps offer structured programs and activ...
601,Yoga Studios,Yoga Studios offer a tranquil space for indivi...


In [15]:
max_length_naics = df_naics["description"].str.len().max()
max_length_business = df_business["description"].str.len().max()

In [16]:
print(f"Max leng naics {max_length_naics}")
print(f"Max leng naics {max_length_business}")

Max leng naics 3594
Max leng naics 838


In [18]:
df_naics["description"].iloc[0]

'Industries in the Crop Production subsector grow crops mainly for food and fiber.  The subsector comprises establishments, such as farms, orchards, groves, greenhouses, and nurseries, primarily engaged in growing crops, plants, vines, or trees and their seeds.\n\nThe industries in this subsector are grouped by similarity of production activity, including biological and physiological characteristics and economic requirements, the length of growing season, degree of crop rotation, extent of input specialization, labor requirements, and capital demands.  The production process is typically completed when the raw product or commodity grown reaches the "farm gate" for market, that is, at the point of first sale or price determination.\n\nEstablishments are classified in the Crop Production subsector when crop production (i.e., value of crops for market) accounts for one-half or more of the establishment\'s total agricultural production.  Within the subsector, establishments are classified 

In [62]:
from openai import OpenAI

openai_api_key = "sk-proj-OSCk0tPAJxvpKtxmZIObT3BlbkFJA9hXFC8sq06DAxMmJgQl"
client = OpenAI(api_key=openai_api_key)

def summarize_openai( description):
    # descriptions = df_naics["description"].iloc[0]

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "user",
                "content":f"Please provide a concise summary of the following industry description. Extract the most relevant information and ensure the summary does not exceed 256 characters: {description}.",
            }
        ],
        # max_tokens = 250
    )

   
    return response.choices[0].message.content

In [65]:
import tqdm

In [63]:
def summarize_description(df, column_name):
    summaries = []

    # Iterate over each row in the DataFrame
    # Iterate over each row in the DataFrame
    for description in tqdm(df[column_name], desc="Processing summaries"):

        # Assuming the API response structure has 'choices' with 'message' containing 'content'
        summary = summarize_openai(description)

        print(f"ORIGINAL: {len(description)}: {description}")
        print(f"SUMMARY:  {len(summary)}: {summary}")

        summaries.append(summary)

    # Return all summaries as a list or add them to the DataFrame
    df["summary"] = summaries

    return df

In [83]:
def keyword_openai(label, description):

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "user",
                "content": f"Could you identify the main keywords from the NAICS taxonomy for a specific industry sector? The industry is named '{label}', and its description is '{description}'. Please return as a list of the maximum 10 keywords, separeted by comma",
            }
        ],
    )

    return response.choices[0].message.content


def extract_keywords(df, label, column_name):

    keywords_list = []
    # Iterate over each row in the DataFrame
    # Iterate over each row in the DataFrame
    for  _,row in df.iterrows():

        # Assuming the API response structure has 'choices' with 'message' containing 'content'
        keywords = keyword_openai(row[label], row[column_name])

        print(f"ORIGINAL: {len(row[column_name])}: {row[column_name]}")
        print(f"Keywords:  {len(keywords)}: {keywords}")

        keywords_list.append(keywords)

    # Return all summaries as a list or add them to the DataFrame
    df["keywords"] = keywords_list

    return df

In [84]:
new_df = summarize_description(df_naics, "description")

ORIGINAL: 1919: Industries in the Crop Production subsector grow crops mainly for food and fiber.  The subsector comprises establishments, such as farms, orchards, groves, greenhouses, and nurseries, primarily engaged in growing crops, plants, vines, or trees and their seeds.

The industries in this subsector are grouped by similarity of production activity, including biological and physiological characteristics and economic requirements, the length of growing season, degree of crop rotation, extent of input specialization, labor requirements, and capital demands.  The production process is typically completed when the raw product or commodity grown reaches the "farm gate" for market, that is, at the point of first sale or price determination.

Establishments are classified in the Crop Production subsector when crop production (i.e., value of crops for market) accounts for one-half or more of the establishment's total agricultural production.  Within the subsector, establishments are c

KeyboardInterrupt: 

In [85]:
new_df = extract_keywords(df_naics, "naics_label", "description")

ORIGINAL: 1919: Industries in the Crop Production subsector grow crops mainly for food and fiber.  The subsector comprises establishments, such as farms, orchards, groves, greenhouses, and nurseries, primarily engaged in growing crops, plants, vines, or trees and their seeds.

The industries in this subsector are grouped by similarity of production activity, including biological and physiological characteristics and economic requirements, the length of growing season, degree of crop rotation, extent of input specialization, labor requirements, and capital demands.  The production process is typically completed when the raw product or commodity grown reaches the "farm gate" for market, that is, at the point of first sale or price determination.

Establishments are classified in the Crop Production subsector when crop production (i.e., value of crops for market) accounts for one-half or more of the establishment's total agricultural production.  Within the subsector, establishments are c

In [68]:
print(new_df["description"].iloc[93])
print(new_df["summary"].iloc[93])

The Administration of Economic Programs subsector groups government establishments primarily engaged in the administration of economic programs.

The Administration of Economic Programs subsector includes government entities that primarily manage economic programs.


In [86]:
new_df

,naics_code,naics_label,description,summary,keywords
0,111,Crop Production,Industries in the Crop Production subsector gr...,The Crop Production subsector comprises establ...,"Crop Production, farms, orchards, greenhouses,..."
1,112,Animal Production and Aquaculture,Industries in the Animal Production and Aquacu...,The Animal Production and Aquaculture subsecto...,"Animal Production, Aquaculture, Ranches, Farms..."
2,113,Forestry and Logging,Industries in the Forestry and Logging subsect...,The forestry and logging industries harvest ti...,"Forestry, Logging, Timber Production, Reforest..."
3,114,"Fishing, Hunting and Trapping","Industries in the Fishing, Hunting and Trappin...","The Fishing, Hunting and Trapping industry rel...","Fishing, Hunting, Trapping, Harvest, Wild Anim..."
4,115,Support Activities for Agriculture and Forestry,Industries in the Support Activities for Agric...,The Support Activities for Agriculture and For...,"Support Activities, Agriculture, Forestry, Sup..."
...,...,...,...,...,...
91,924,Administration of Environmental Quality Programs,The Administration of Environmental Quality Pr...,The Administration of Environmental Quality Pr...,"Administration, Environmental Quality Programs..."
92,925,"Administration of Housing Programs, Urban Plan...","The Administration of Housing Programs, Urban ...","The Administration of Housing Programs, Urban ...","Administration, Housing Programs, Urban Planni..."
93,926,Administration of Economic Programs,The Administration of Economic Programs subsec...,The Administration of Economic Programs subsec...,"Administration, Economic Programs, Subsector, ..."
94,927,Space Research and Technology,The Space Research and Technology subsector co...,The Space Research and Technology subsector pr...,"Space Research, Technology, Government Establi..."


In [82]:
new_df["description"].loc[0]

'Industries in the Crop Production subsector grow crops mainly for food and fiber.  The subsector comprises establishments, such as farms, orchards, groves, greenhouses, and nurseries, primarily engaged in growing crops, plants, vines, or trees and their seeds.\n\nThe industries in this subsector are grouped by similarity of production activity, including biological and physiological characteristics and economic requirements, the length of growing season, degree of crop rotation, extent of input specialization, labor requirements, and capital demands.  The production process is typically completed when the raw product or commodity grown reaches the "farm gate" for market, that is, at the point of first sale or price determination.\n\nEstablishments are classified in the Crop Production subsector when crop production (i.e., value of crops for market) accounts for one-half or more of the establishment\'s total agricultural production.  Within the subsector, establishments are classified 

In [87]:
new_df.to_csv("naics_summary_keywords.csv")

In [23]:
df_naics

,naics_code,naics_label,description
0,111,Crop Production,Industries in the Crop Production subsector gr...
1,112,Animal Production and Aquaculture,Industries in the Animal Production and Aquacu...
2,113,Forestry and Logging,Industries in the Forestry and Logging subsect...
3,114,"Fishing, Hunting and Trapping","Industries in the Fishing, Hunting and Trappin..."
4,115,Support Activities for Agriculture and Forestry,Industries in the Support Activities for Agric...
...,...,...,...
91,924,Administration of Environmental Quality Programs,The Administration of Environmental Quality Pr...
92,925,"Administration of Housing Programs, Urban Plan...","The Administration of Housing Programs, Urban ..."
93,926,Administration of Economic Programs,The Administration of Economic Programs subsec...
94,927,Space Research and Technology,The Space Research and Technology subsector co...
